In [ ]:
# this is largely based on the tutorials at: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import datetime
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from sklearn.metrics import mean_squared_error

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
# settings
model_settings = {'epochs': 100, 'batch_size': 4, 'train_test_ratio': 0.7, 'hidden_layers': 3, 'units': 100, 'start_date': '2020-01-01', 'n_steps_in': 60, 'n_steps_out': 30, 'symbol': 'CTXR'}
current_date = datetime.datetime.now().strftime("%Y-%m-%d")


In [3]:
# load and shape data
conn = sqlite3.connect('stockPrediction_06072021.db')

symbol_info = conn.execute(f"SELECT sector_id, industry_id FROM stock WHERE stock_symbol = \"{model_settings['symbol']}\";")
symbol_info = symbol_info.fetchall()
sector_id = symbol_info[0][0]
industry_id = symbol_info[0][1]

query = f"SELECT r.stock_symbol, l.price_datetime, l.open_price, l.high_price, l.low_price, l.close_price, l.volume, l.dividends, l.stock_splits FROM eod_price_history l INNER JOIN stock r ON r.stock_id = l.stock_id WHERE r.sector_id = {sector_id} OR r.industry_id = {industry_id};"

symbols = conn.execute('SELECT stock_symbol FROM stock')
symbols = symbols.fetchall()
symbols = [i[0] for i in symbols]
symbols = [i for i in symbols if i not in symbols]

df = pd.read_sql(query, conn, index_col=['stock_symbol', 'price_datetime'])
df = df.reset_index()

df['price_datetime'] = pd.to_datetime(df['price_datetime'], format='%Y-%m-%d')

df = df.set_index(['price_datetime', 'stock_symbol']).unstack(['stock_symbol'])

df = df.loc[model_settings['start_date']:current_date]  # date range from 2019-01-01 to 2021-05-31

close_df = df['close_price'].dropna(thresh=(len(df['close_price'] / 0.2)), axis=1)

close_df = close_df.fillna(method='ffill', axis=1)

# remove outliers
low_outlier = close_df.quantile(.1, axis=1).quantile(.1)
high_outlier = close_df.quantile(.9, axis=1).quantile(.9)
for column in close_df.columns:
    if (close_df[column].median() < low_outlier) or (close_df[column].median() > high_outlier):
        close_df = close_df.drop([column], axis=1)
close_df

stock_symbol,ABEO,ABIO,ABMC,ABMT,ABUS,ACER,ACHFF,ACRX,ACST,ACUR,ADIL,ADMA,ADMP,ADMS,ADMT,ADXS,AEMD,AEZS,AGE,AGEN,AGRX,AGTC,AHPI,AIKI,AIM,AKBA,AKTX,ALEAF,ALID,ALIM,ALNA,ALRN,AMPE,AMRN,AMRX,AMS,ANIX,APEN,APM,APOP,...,TRTC,TRVI,TRVN,TTNP,TTOO,TXMD,TYHT,TYME,VBIO,VBIV,VBLT,VCNX,VEGGF,VERO,VEXTF,VIVE,VIVXF,VLNCF,VNRX,VRAY,VSTM,VTGN,VTVT,VVCIF,WDDMF,WKULF,WORX,XBIO,XCUR,XERS,XPHYF,XTLB,XTNT,XXII,YCBD,ZIOP,ZIVO,ZOM,ZSAN,ZYNE
price_datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3.210,5.6800,0.0700,0.10000,2.94,3.790,0.7410,2.070,2.3800,0.280,2.4400,3.935,0.7500,4.080,0.190,1.0900,1.2500,0.8800,1.9500,4.040,2.9700,4.15,1.26,1.4000,0.6100,6.340,1.740,0.4750,1.55,8.160,2.8900,0.595,0.5900,21.530001,4.7700,2.5500,3.000,2.9200,13.6600,2.390,...,0.17000,3.750,0.858,6.6000,1.160,2.440,5.6250,1.4200,0.050,1.3400,1.2500,5.01,0.15180,4.550,0.55930,11.0000,0.4316,2.73000,4.670,4.055,1.330,0.7500,1.775,0.16200,0.6743,0.21000,2.921,1.4000,2.800,6.70,0.75,1.3000,1.580,1.140,2.3400,4.6200,0.160,0.3310,1.5200,5.8800
2020-01-03,2.900,5.5600,0.0700,0.10000,3.08,3.580,0.7410,2.040,2.3300,0.280,2.4400,3.760,0.8400,4.100,0.200,1.1200,1.1500,0.8800,1.9000,3.860,2.9000,4.18,1.24,1.3200,0.5600,6.400,1.710,0.4640,1.55,7.590,2.7800,0.605,0.6000,20.950001,4.7200,2.4300,3.160,2.8600,14.7300,2.475,...,0.17000,4.284,0.859,7.8000,1.230,2.310,5.4900,1.3900,0.050,1.3200,1.2600,4.94,0.15500,4.480,0.55216,10.1000,0.4400,2.78000,4.800,4.010,1.300,0.7300,1.830,0.15800,0.6516,0.19375,2.900,1.4600,2.830,6.07,0.75,1.4100,1.530,1.090,2.2700,4.5000,0.160,0.3320,1.5500,5.8100
2020-01-06,2.770,5.6300,0.0700,0.10000,3.18,3.690,0.7410,2.010,2.2100,0.280,2.4200,3.670,0.8900,4.140,0.190,1.0700,1.1200,0.9200,1.7800,3.780,2.8500,4.13,1.25,1.3000,0.5600,6.620,1.700,0.4440,1.55,7.760,2.7000,0.649,0.6900,21.230000,4.7300,2.5300,3.160,2.9000,14.7500,2.410,...,0.16000,4.372,0.861,9.0000,1.190,2.340,5.9400,1.5400,0.050,1.4200,1.2500,4.99,0.19800,4.600,0.56000,10.1000,0.4474,2.65000,5.160,3.880,1.310,0.7300,1.890,0.16900,0.6249,0.21625,2.920,1.5200,2.860,5.79,0.75,1.4200,1.550,1.100,2.2200,4.4900,0.160,0.3130,1.6400,5.7000
2020-01-07,2.570,5.6700,0.0800,0.10000,3.04,3.710,0.7250,2.070,2.3300,0.240,2.3900,3.760,0.8200,4.160,0.180,1.3000,1.1500,0.9700,1.8000,3.710,2.8500,4.27,1.27,1.3300,0.6800,6.820,1.740,0.4400,1.55,6.677,2.5700,0.650,0.7300,20.080000,4.6800,2.4000,2.970,2.8000,14.8100,4.090,...,0.16000,4.376,0.880,7.2000,1.190,2.260,5.5530,1.7500,0.050,1.3900,1.2500,4.90,0.21700,5.390,0.62680,10.5000,0.4400,2.50335,4.910,3.760,1.260,0.7200,1.770,0.16700,0.6100,0.25875,3.100,1.4700,2.800,6.39,0.75,1.4100,1.590,1.090,1.8400,4.5300,0.160,0.3130,1.7100,5.5600
2020-01-08,2.620,6.0700,0.0800,0.10000,3.07,3.690,0.7330,2.030,2.1900,0.240,2.4700,3.720,0.8000,5.110,0.200,1.2300,1.2500,1.1100,1.9100,3.650,2.9200,4.16,1.32,1.3100,0.6500,7.150,1.770,0.4450,1.55,6.610,2.4900,0.605,0.7200,19.459999,4.5200,2.4200,3.000,2.9100,14.7500,3.240,...,0.17000,4.410,0.859,6.6000,1.150,2.310,5.5080,1.6700,0.040,1.3600,1.2400,4.91,0.19860,4.970,0.47040,9.8900,0.4296,2.52000,4.950,3.810,1.720,0.7000,2.040,0.17000,0.6100,0.25000,2.950,1.4900,2.580,6.25,0.75,1.2600,1.580,1.070,1.5900,4.5900,0.160,0.3200,1.6200,5.3100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,1.710,3.4900,0.0636,0.07920,2.86,2.960,1.1180,1.430,0.5071,0.365,2.5700,1.750,0.6991,5.240,0.139,0.4800,2.0000,0.9012,1.2555,4.315,1.6100,4.05,3.72,1.0807,2.0899,3.635,1.790,0.3581,1.03,10.500,1.1087,1.250,1.8208,4.570000,5.7100,2.5800,4.020,7.2376,2.8299,3.375,...,0.19720,2.100,1.800,2.4300,1.270,1.250,6.8800,1.6500,0.160,3.3300,2.4400,2.42,0.06355,2.100,0.75000,2.7700,0.3820,2.61000,3.510,5.980,3.950,2.6000,2.400,0.116

In [4]:
# split sequences function, splits multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find end of patterns
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check to see if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input, output parts of the pattern
        # slightly guessing here on how to combine these two examples
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [5]:
# split into test/train data sets

# splitting test/training data
data_size = len(close_df)

# using a 90/10 train/test split
training_data = close_df.iloc[:(int(data_size * model_settings['train_test_ratio']))]
test_data = close_df.iloc[(int(data_size * model_settings['train_test_ratio'])):]

In [ ]:
# testing data sets to see if they're correct
training_data

In [ ]:
test_data

In [6]:
#df['sum'] = df.sum(axis=1)

# ignore others, working w/ this cell currently

arrays = list()
out_seq = list()
for i in range(len(training_data)):
    arrays.append(training_data.iloc[i].to_numpy(dtype=np.float))
'''for i in range(len(training_data)):
    out_seq.append(training_data.iloc[i].sum())
out_seq = np.array(out_seq, dtype=np.float)'''

'for i in range(len(training_data)):\n    out_seq.append(training_data.iloc[i].sum())\nout_seq = np.array(out_seq, dtype=np.float)'

In [18]:
len(arrays[0])

412

In [7]:
# reshape arrays
# in_seq arrays
for i in range(len(arrays)):
    arrays[i] = arrays[i].reshape((len(arrays[i]), 1))
# out_seq = out_seq.reshape((len(out_seq), 1))

In [8]:
# arrays.append(out_seq)
dataset = np.hstack((arrays))

In [9]:
# may just be able to jump right into using the split_sequence func?
X, y = split_sequences(dataset, model_settings['n_steps_in'], model_settings['n_steps_out'])

In [19]:
for i in range(len(y)):
    print(y[i].shape)

(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)
(30, 251)


In [10]:
n_features = X.shape[2]

In [11]:
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [13]:
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(model_settings['n_steps_in'], n_features)))
model.add(RepeatVector(model_settings['n_steps_out']))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
# model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

In [14]:
# fit model
model.fit(X, y, epochs=400, verbose=1)

Epoch 1/400
11/11 [==============================] - 4s 220ms/step - loss: nan
Epoch 2/400
11/11 [==============================] - 3s 228ms/step - loss: nan
Epoch 3/400
11/11 [==============================] - 2s 220ms/step - loss: nan
Epoch 4/400
11/11 [==============================] - 3s 230ms/step - loss: nan
Epoch 5/400
11/11 [==============================] - 2s 227ms/step - loss: nan
Epoch 6/400
11/11 [==============================] - 2s 225ms/step - loss: nan
Epoch 7/400
11/11 [==============================] - 2s 226ms/step - loss: nan
Epoch 8/400
11/11 [==============================] - 2s 224ms/step - loss: nan
Epoch 9/400
 5/11 [============>.................] - ETA: 1s - loss: nan

KeyboardInterrupt: 

In [20]:
X.shape, y.shape

((323, 60, 251), (323, 30, 251))

In [25]:
len(training_data)

251